In [51]:
import numpy as np
np.random.seed(42)
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,confusion_matrix, f1_score, precision_score, recall_score

from keras.models import Model
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras import backend as K
import warnings
warnings.filterwarnings('ignore')

import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from wordcloud import WordCloud, STOPWORDS 
import plotly.graph_objs as go
from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import scipy.stats as ss
import matplotlib_venn as venn
import re
#from plotly.offline import plot


#misc
import gc
import time
import warnings
import matplotlib.gridspec as gridspec
#nlp
import string
import re    #for regex
import nltk
from nltk.corpus import stopwords
import spacy
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
# Tweet tokenizer does not split at apostophes which is what we want
from nltk.tokenize import TweetTokenizer   


#FeatureEngineering
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_is_fitted
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split


tokenizer=TweetTokenizer()
from nltk.stem.wordnet import WordNetLemmatizer 
nltk.download('wordnet')
lem = WordNetLemmatizer()

color = sns.color_palette()


import string,re
from nltk.corpus import stopwords
eng_stopwords = set(stopwords.words("english"))


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jixusheng/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [52]:
eda_columns=['article_id', 'article_id_flag', 'clean', 'clean_comment_text',
       'comment_text', 'count_ip', 'count_links', 'id', 'identity_hate',
       'insult', 'ip', 'label_num', 'link', 'num_char', 'num_char_norm',
       'num_words', 'num_words_norm', 'obscene', 'punctuation',
       'punctuation_norm', 'sent_num', 'sent_num_norm', 'severe_toxic',
       'stop_words', 'stop_words_norm', 'threat', 'title', 'title_norm',
       'toxic', 'unique', 'unique_norm', 'upper', 'upper_norm', 'username',
       'username_nums']
non_eda_columns=['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate', 'clean', 'label_num', 'sent_num',
       'sent_num_norm', 'num_words', 'num_words_norm', 'num_char',
       'num_char_norm', 'stop_words', 'stop_words_norm', 'punctuation',
       'punctuation_norm', 'upper', 'upper_norm', 'title', 'title_norm',
       'unique', 'unique_norm', 'ip', 'count_ip', 'link', 'count_links',
       'article_id', 'article_id_flag', 'username', 'username_nums',
       'clean_comment_text']
test_columns=['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate', 'comment_text', 'sent_num', 'sent_num_norm',
       'num_words', 'num_words_norm', 'num_char', 'num_char_norm',
       'stop_words', 'stop_words_norm', 'punctuation', 'punctuation_norm',
       'upper', 'upper_norm', 'title', 'title_norm', 'unique', 'unique_norm',
       'ip', 'count_ip', 'link', 'count_links', 'article_id',
       'article_id_flag', 'username', 'username_nums', 'clean_comment_text']

train=pd.read_csv("/Users/jixusheng/Documents/kaggle/eda_nlp-master/non_eda_train.txt",names=non_eda_columns,header=0)
eda_train=pd.read_csv("/Users/jixusheng/Documents/kaggle/eda_nlp-master/eda_train.txt",names=eda_columns,header=0)
test=pd.read_csv("/Users/jixusheng/Documents/kaggle/eda_nlp-master/new_test_labels.txt",names=test_columns,header=0)

# train=train.head(1000)
# test=test.head(1000)

In [53]:
len(train)

159571

In [110]:
test['clean']=test['toxic']+test['severe_toxic']+test['obscene']+test['threat']+test['insult']+test['identity_hate']
test['clean']=test['clean'].apply(lambda x:1 if x==0 else 0)

In [54]:
train[['clean','clean_comment_text','id']].to_csv('/Users/jixusheng/Documents/kaggle/eda_nlp-master/input.txt', sep='\t', index=False)

In [81]:
train[train['id']=="0bed2196c873636d"]['comment_text']

4482    1993\n\n1994\n\n1995\n\n1996\n\n1997\n\n1998\n...
Name: comment_text, dtype: object

In [56]:
#https://drive.google.com/file/d/0B1yuv8YaUVlZZ1RzMFJmc1ZsQmM/view
# Aphost lookup dict
APPO = {
"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"tryin'":"trying"
}

In [76]:
def clean(comment):
    """
    This function receives comments and returns clean word-list
    """
    # Convert to lower case , so that Hi and hi are the same
    comment = comment.lower()

    # remove \n
    comment = re.sub("\\n", "", comment)

    comment = re.sub("\:", "", comment)

    comment = re.sub("\==", "", comment)

    #     # remove leaky elements like ip,user
    comment = re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", "", comment)
    # removing usernames
    comment = re.sub("\[\[User(.*)\]", "", comment)

    # Split the sentences into words
    words = tokenizer.tokenize(comment)

    #     # (')aphostophe  replacement (ie)   you're --> you are
    #     # ( basic dictionary lookup : master dictionary present in a hidden block of code)
    words = [APPO[word] if word in APPO else word for word in words]
    words = [lem.lemmatize(word, "v") for word in words]
    #words = [w for w in words if not w in eng_stopwords]
    #print (words)
    clean_sent = " ".join(words)
    # remove any non alphanum,digit character
    clean_sent = re.sub("\W+", " ", clean_sent)

    clean_sent = re.sub("  ", " ", clean_sent)
    clean_sent = re.sub("\d+", " ", clean_sent)
    clean_sent =re.sub('[^0-9a-zA-Z]+', " ", s)
    return (clean_sent)

In [107]:
clean_sent="1993\n\n1994\n\n1995\n\n1996\n\n1997\n\n1998\n..."
#clean_sent=re.sub("\W+", " ", clean_sent)
clean_sent=re.sub('[^a-zA-Z]+', " ",clean_sent)
clean_sent



' '

In [108]:
a=bool(re.search('[a-zA-Z]', clean_sent))
a

False

In [77]:
train['clean_comment_text']=train['comment_text'].apply(lambda x :clean(x))

In [78]:
test['clean_comment_text']=test['comment_text'].apply(lambda x :clean(x))

In [118]:
train[['clean','clean_comment_text','id']].to_csv('/Users/jixusheng/Documents/kaggle/eda_nlp-master/input.txt', sep='\t', index=False)

In [111]:
test[['clean','clean_comment_text','id']].to_csv('/Users/jixusheng/Documents/kaggle/eda_nlp-master/input.txt', sep='\t', index=False)

In [112]:
colnames=['clean', 'clean_comment_text','id'] 
test_new_eda = pd.read_csv('/Users/jixusheng/Documents/kaggle/eda_nlp-master/out.txt', sep='\t', header=None,names=colnames)


In [120]:
train = pd.read_csv('/Users/jixusheng/Documents/kaggle/eda_nlp-master/out.txt', sep='\t', header=None,names=colnames)



In [125]:
len(train)

797820

In [124]:
eda_columns=['article_id', 'article_id_flag', 'clean', 'clean_comment_text',
       'comment_text', 'count_ip', 'count_links', 'id', 'identity_hate',
       'insult', 'ip', 'label_num', 'link', 'num_char', 'num_char_norm',
       'num_words', 'num_words_norm', 'obscene', 'punctuation',
       'punctuation_norm', 'sent_num', 'sent_num_norm', 'severe_toxic',
       'stop_words', 'stop_words_norm', 'threat', 'title', 'title_norm',
       'toxic', 'unique', 'unique_norm', 'upper', 'upper_norm', 'username',
       'username_nums']
non_eda_columns=['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate', 'clean', 'label_num', 'sent_num',
       'sent_num_norm', 'num_words', 'num_words_norm', 'num_char',
       'num_char_norm', 'stop_words', 'stop_words_norm', 'punctuation',
       'punctuation_norm', 'upper', 'upper_norm', 'title', 'title_norm',
       'unique', 'unique_norm', 'ip', 'count_ip', 'link', 'count_links',
       'article_id', 'article_id_flag', 'username', 'username_nums',
       'clean_comment_text']
test_columns=['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate', 'comment_text', 'sent_num', 'sent_num_norm',
       'num_words', 'num_words_norm', 'num_char', 'num_char_norm',
       'stop_words', 'stop_words_norm', 'punctuation', 'punctuation_norm',
       'upper', 'upper_norm', 'title', 'title_norm', 'unique', 'unique_norm',
       'ip', 'count_ip', 'link', 'count_links', 'article_id',
       'article_id_flag', 'username', 'username_nums', 'clean_comment_text']

train_old=pd.read_csv("/Users/jixusheng/Documents/kaggle/eda_nlp-master/non_eda_train.txt",names=non_eda_columns,header=0)
#eda_train=pd.read_csv("/Users/jixusheng/Documents/kaggle/eda_nlp-master/eda_train.txt",names=eda_columns,header=0)
test_old=pd.read_csv("/Users/jixusheng/Documents/kaggle/eda_nlp-master/new_test_labels.txt",names=test_columns,header=0)

# train=train.head(1000)
# test=test.head(1000)

In [126]:
train_new_eda=pd.merge(train,train_old,how="left",on='id')


In [130]:
test_new_eda=pd.merge(test,test_old,how="left",on='id')



In [129]:
train_new_eda.head(3)

clean_x                               clean_comment_text_x  \
0        1  explanationwhy the edit make under my username...   
1        1  explanationwhy the edit make under my username...   
2        1  explanationwhy the edit make under my username...   

                 id                                       comment_text  toxic  \
0  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
1  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
2  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   

   severe_toxic  obscene  threat  insult  identity_hate  ...  unique_norm  \
0             0        0       0       0              0  ...      0.04908   
1             0        0       0       0              0  ...      0.04908   
2             0        0       0       0              0  ...      0.04908   

                 ip  count_ip  link  count_links  article_id  article_id_flag  \
0  ['89.205.38.27']         1    []            0          []                0   
1  ['89.205.38.27']         1    []            0          []                0   
2  ['89.205.38.27']         1    []            0          []                0   

   username  username_nums                               clean_comment_text_y  
0        []              0  explanationwhy edit make username hardcore met...  
1        []              0  explanationwhy edit make username hardcore met...  
2        []              0  explanationwhy edit make username hardcore met...  

[3 rows x 37 columns]

In [131]:
test_new_eda.head(3)

clean                               clean_comment_text_x                id  \
0      1  not you for understand i think very highly of ...  0001ea8717f6de06   
1      1  thank you for understand i think very highly o...  0001ea8717f6de06   
2      1  thank you for understand i think very highly o...  0001ea8717f6de06   

   toxic  severe_toxic  obscene  threat  insult  identity_hate  \
0      0             0        0       0       0              0   
1      0             0        0       0       0              0   
2      0             0        0       0       0              0   

                                        comment_text  ...  unique_norm  ip  \
0  Thank you for understanding. I think very high...  ...     0.017073  []   
1  Thank you for understanding. I think very high...  ...     0.017073  []   
2  Thank you for understanding. I think very high...  ...     0.017073  []   

   count_ip  link  count_links  article_id  article_id_flag  username  \
0         0    []            0          []                0        []   
1         0    []            0          []                0        []   
2         0    []            0          []                0        []   

   username_nums                               clean_comment_text_y  
0              0  thank understand think highly would revert wit...  
1              0  thank understand think highly would revert wit...  
2              0  thank understand think highly would revert wit...  

[3 rows x 35 columns]

In [134]:
train_new_eda=train_new_eda.rename(columns={"clean_comment_text_x":"clean_comment_text"})
test_new_eda=test_new_eda.rename(columns={"clean_comment_text_x":"clean_comment_text"})




clean_x                                 clean_comment_text  \
0        1  explanationwhy the edit make under my username...   
1        1  explanationwhy the edit make under my username...   
2        1  explanationwhy the edit make under my username...   

                 id                                       comment_text  toxic  \
0  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
1  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
2  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   

   severe_toxic  obscene  threat  insult  identity_hate  ...  unique_norm  \
0             0        0       0       0              0  ...      0.04908   
1             0        0       0       0              0  ...      0.04908   
2             0        0       0       0              0  ...      0.04908   

                 ip  count_ip  link  count_links  article_id  article_id_flag  \
0  ['89.205.38.27']         1    []            0          []                0   
1  ['89.205.38.27']         1    []            0          []                0   
2  ['89.205.38.27']         1    []            0          []                0   

   username  username_nums                               clean_comment_text_y  
0        []              0  explanationwhy edit make username hardcore met...  
1        []              0  explanationwhy edit make username hardcore met...  
2        []              0  explanationwhy edit make username hardcore met...  

[3 rows x 37 columns]

In [135]:
test_new_eda.to_csv('/Users/jixusheng/Documents/kaggle/eda_nlp-master/new_eda_test.txt', index=False)

In [136]:
train_new_eda.to_csv('/Users/jixusheng/Documents/kaggle/eda_nlp-master/new_eda_train.txt', index=False)

In [141]:
test_new_eda.columns

Index(['clean', 'clean_comment_text', 'id', 'toxic', 'severe_toxic', 'obscene',
       'threat', 'insult', 'identity_hate', 'comment_text', 'sent_num',
       'sent_num_norm', 'num_words', 'num_words_norm', 'num_char',
       'num_char_norm', 'stop_words', 'stop_words_norm', 'punctuation',
       'punctuation_norm', 'upper', 'upper_norm', 'title', 'title_norm',
       'unique', 'unique_norm', 'ip', 'count_ip', 'link', 'count_links',
       'article_id', 'article_id_flag', 'username', 'username_nums',
       'clean_comment_text_y'],
      dtype='object')

In [139]:
c=['clean_x', 'clean_comment_text', 'id', 'comment_text', 'toxic',
       'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate',
       'clean_y', 'label_num', 'sent_num', 'sent_num_norm', 'num_words',
       'num_words_norm', 'num_char', 'num_char_norm', 'stop_words',
       'stop_words_norm', 'punctuation', 'punctuation_norm', 'upper',
       'upper_norm', 'title', 'title_norm', 'unique', 'unique_norm', 'ip',
       'count_ip', 'link', 'count_links', 'article_id', 'article_id_flag',
       'username', 'username_nums', 'clean_comment_text_y']
train=pd.read_csv("/Users/jixusheng/Documents/kaggle/eda_nlp-master/new_eda_train.txt",names=c,header=0)




In [142]:
c_t=['clean', 'clean_comment_text', 'id', 'toxic', 'severe_toxic', 'obscene',
       'threat', 'insult', 'identity_hate', 'comment_text', 'sent_num',
       'sent_num_norm', 'num_words', 'num_words_norm', 'num_char',
       'num_char_norm', 'stop_words', 'stop_words_norm', 'punctuation',
       'punctuation_norm', 'upper', 'upper_norm', 'title', 'title_norm',
       'unique', 'unique_norm', 'ip', 'count_ip', 'link', 'count_links',
       'article_id', 'article_id_flag', 'username', 'username_nums',
       'clean_comment_text_y']
test=pd.read_csv("/Users/jixusheng/Documents/kaggle/eda_nlp-master/new_eda_test.txt",names=c_t,header=0)

In [145]:
len(train)

797820